# TODO

1. Run the TF-IDF with each of the program name
2. Replace vec with original words
3. Run Cosine to get the most needed skills
4. Check why the tf-idf score is off!!!

### TF IDF preporccesing


In [1]:
import os

import pandas as pd
from ast import literal_eval
# from scipy.stats import f_oneway
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# Ensure the project directory is set correctly
if not set(["notebooks", "scripts"]).issubset(os.listdir()):
    os.chdir(os.path.dirname(os.getcwd()))

In [3]:
# from scripts import preprocessing_helper

In [4]:
# Define file paths
job_desc_filename = "data/interim/coop_postings_program.csv"
foundal_skills_filename = "data/interim/soft_skills.csv"
post_proc_job_post = "data/interim/post_processing_job_postings.csv"
post_proc_found_skills = "data/interim/post_processing_soft_skills.csv"
replace_skills_job_post = "data/interim/replace_skills_job_programs.csv"

In [5]:
# Check if running in Google Colab
try:
    import google.colab

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [6]:
# Mount Google Drive and adjust file paths if in Google Colab
if IN_COLAB:
    drive_mounted_path = "/content/drive"

    if not os.path.exists(drive_mounted_path):
        from google.colab import drive

        drive.mount(drive_mounted_path)

    project_dir = "My Drive/School/UoGuelph/MSc/Research/soft_skills_job_desc/jfraeysd"
    job_desc_filename = os.path.join(drive_mounted_path, project_dir, job_desc_filename)
    foundal_skills_filename = os.path.join(
        drive_mounted_path, project_dir, foundal_skills_filename
    )

# Pre-processing job description

In [7]:
# Read job description data
# df_coop = pd.read_csv(job_desc_filename)

In [8]:
# df_coop["job_description_processed"] = df_coop["job_description"].progress_apply(preprocessing_helper.preprocess_text)

In [9]:
# df_coop.to_csv(post_proc_job_post, index=False)

In [10]:
df_coop = pd.read_csv(post_proc_job_post)

In [11]:
df_coop.columns

Index(['job_id', 'job_position', 'date_goes_live', 'job_description',
       'program_name', 'job_description_processed'],
      dtype='object')

# Reading foundational skills

In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
# df_soft_skills = pd.read_csv(foundal_skills_filename)

In [14]:
# df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill'].apply(lambda x: x.split())
# df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels'].apply(lambda x: x.split(", "))

In [15]:
# df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill_lemm'].apply(preprocessing_helper.preprocess_labels)
# df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels_lemm'].apply(preprocessing_helper.preprocess_labels)

In [16]:
# df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill_lemm'].apply(' '.join)
# df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels_lemm'].apply(lambda x: list(set(x)))

In [17]:
# df_soft_skills.to_csv(post_proc_found_skills, index=False)

In [18]:
df_soft_skills = pd.read_csv(post_proc_found_skills)

In [19]:
df_soft_skills['alt_labels_lemm']=df_soft_skills['alt_labels_lemm'].apply(literal_eval)

# Replacing Alternate Labels with Foundational Skills

In [20]:
word_mapping = dict(zip(df_soft_skills['soft_skill_lemm'], df_soft_skills['alt_labels_lemm']))

In [21]:
# df_coop['job_description_replaced_skills']=df_coop['job_description_processed'].progress_apply(lambda x: preprocessing_helper.replace_keywords(x, word_mapping))

In [22]:
# df_coop.to_csv(replace_skills_job_post, index=False)

In [23]:
df_coop = pd.read_csv(replace_skills_job_post)

## Analysis

In [33]:
# from sklearn.feature_extraction.text import CountVectorizer

# Analysis
df_coop_processed = df_coop.copy()
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    norm='l2',
)

tfidf_joined = []
unique_program_names = df_coop_processed["program_name"].unique()

for program_name in tqdm(unique_program_names, desc="Running TF-IDF analysis"):
    df_program_name = df_coop_processed[
        df_coop_processed["program_name"] == program_name
    ]
    tfidf_matrix = tfidf_vectorizer.fit_transform(
        df_program_name["job_description_replaced_skills"]
    )
    df_tfidf = pd.DataFrame(
        tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
    )
    keyword_weights = df_tfidf.mean(axis=0).reset_index()
    keyword_weights.columns = ["keyword", "tf_idf_weight"]
    sorted_keyword_weights = keyword_weights.sort_values(
        by="tf_idf_weight", ascending=False
    )
    tfidf_joined.append((program_name, sorted_keyword_weights))

Running TF-IDF analysis:   0%|          | 0/9 [00:00<?, ?it/s]

In [34]:
df_tfidf = []
for prg_name, df in tfidf_joined:
    df['program_name'] = prg_name
    df_tfidf.append(df)
    
df_tfidf = pd.concat(df_tfidf)

In [35]:
# Assuming df_soft_skillsuniqueins the list of all unique foundational skills
all_foundational_skills = df_soft_skills["soft_skill_lemm"].tolist()

# Create a DataFrame with all soft skills
all_skills_df = pd.DataFrame({"keyword": all_foundational_skills})

# Merge with filtered_tfidf_df to transfer values
filtered_tfidf_df = pd.merge(all_skills_df, df_tfidf, on="keyword", how="left")

# # If there are still NaN values after merging, fill them with zeros
# filtered_tfidf_df.fillna(0, inplace=True)

['accountability' 'active listen' 'adaptive' 'analytical' 'argumentation'
 'coach' 'commitment' 'communication' 'conceptual' 'conflict management'
 'coordination' 'creativity' 'critical think' 'curiosity' 'decision'
 'decision make' 'detail' 'diverse' 'eagerness' 'emotional intelligence'
 'enthusiasm' 'ethic' 'flexibility' 'goal' 'hospitality' 'impartiality'
 'influence' 'initiative' 'integrity' 'interpersonal communication'
 'kindness' 'leadership' 'mentor' 'motivate' 'negotiation' 'optimism'
 'oral communication' 'passion' 'persuasion' 'presentation'
 'problem solve' 'self confidence' 'self organize' 'social skill' 'speak'
 'strategic think']


In [36]:
# filtered_tfidf_joined = [
#     pd.concat([
#         sorted_df[sorted_df["keyword"].isin([soft_skill])]
#         for soft_skill in df_soft_skills["soft_skill"]
#     ], ignore_index=True).assign(program_name=program_name)
#     for program_name, sorted_df in tfidf_joined
# ]

# # Create a DataFrame
# filtered_tfidf_df = pd.concat(filtered_tfidf_joined, ignore_index=True)

In [37]:
filtered_tfidf_df.groupby('program_name').get_group('Computer Science').sort_values('tf_idf_weight', ascending=False)

,keyword,tf_idf_weight,program_name
15,active listen,0.016226,Computer Science
132,decision,0.012227,Computer Science
213,goal,0.011825,Computer Science
141,decision make,0.011230,Computer Science
6,accountability,0.010684,Computer Science
105,creativity,0.010323,Computer Science
407,strategic think,0.007571,Computer Science
33,analytical,0.007269,Computer Science
123,curiosity,0.007227,Computer Science
321,optimism,0.007202,Computer Science


In [38]:
# Transpose the DataFrame
transposed_tfidf_df = filtered_tfidf_df.pivot(index='program_name', columns='keyword', values='tf_idf_weight')

In [39]:
transposed_tfidf_df = transposed_tfidf_df[transposed_tfidf_df.index.notnull()]

In [44]:
# Sort the pivot table by TF-IDF averages (mean across years)
pivot_table_sorted = transposed_tfidf_df.mean(axis=0).sort_values(ascending=False).index

# Reorder the rows in the pivot table and replace NaN with 0.00
transposed_tfidf_df = transposed_tfidf_df.reindex(columns=pivot_table_sorted).fillna(0.00)
transposed_tfidf_df

keyword,active listen,goal,decision,accountability,decision make,creativity,curiosity,analytical,communication,optimism,...,eagerness,speak,influence,kindness,hospitality,conflict management,social skill,persuasion,interpersonal communication,self organize
program_name,,,,,,,,,,,,,,,,,,,,,
Biological Engineering,0.012973,0.011720,0.009882,0.011509,0.009228,0.006760,0.005848,0.006772,0.006350,0.005028,...,0.000866,0.000930,0.000583,0.000662,0.000228,0.000187,0.000134,0.000036,0.000016,0.000000
Biomedical Engineering,0.013086,0.011435,0.010587,0.011905,0.010008,0.007448,0.006043,0.007150,0.006495,0.005091,...,0.000865,0.001007,0.000623,0.000773,0.000197,0.000228,0.000146,0.000040,0.000025,0.000000
Computer Engineering,0.015273,0.011596,0.012185,0.010739,0.011308,0.009794,0.006851,0.006581,0.006000,0.006649,...,0.001467,0.001227,0.000943,0.000673,0.000940,0.000122,0.000083,0.000078,0.000051,0.000029
Computer Science,0.016226,0.011825,0.012227,0.010684,0.011230,0.010323,0.007227,0.007269,0.006034,0.007202,...,0.001570,0.001280,0.001114,0.000667,0.001117,0.000130,0.000067,0.000090,0.000045,0.000034
Engineering Systems & Computing,0.014964,0.011528,0.012165,0.010786,0.011319,0.009656,0.006717,0.006363,0.005967,0.006474,...,0.001433,0.001209,0.000920,0.000698,0.000881,0.000125,0.000082,0.000066,0.000064,0.000026
Environmental Engineering,0.013426,0.012820,0.010764,0.011233,0.010013,0.006301,0.006122,0.005646,0.006163,0.005395,...,0.000799,0.000999,0.000562,0.000535,0.000259,0.000213,0.000136,0.000033,0.000044,0.000000
Mechanical Engineering,0.013575,0.012253,0.012417,0.011896,0.011760,0.007987,0.006039,0.005189,0.006459,0.005557,...,0.001086,0.001071,0.000598,0.000755,0.000283,0.000243,0.000107,0.000054,0.000070,0.000010
Software Engineering,0.016146,0.011837,0.012427,0.010692,0.011456,0.010391,0.007200,0.007229,0.006234,0.007295,...,0.001575,0.001300,0.001120,0.000668,0.001125,0.000128,0.000067,0.000084,0.000034,0.000035
Water Resources Engineering,0.013414,0.012871,0.010876,0.011261,0.010081,0.006339,0.006132,0.005577,0.006179,0.005330,...,0.000795,0.001012,0.000569,0.000528,0.000240,0.000216,0.000140,0.000030,0.000032,0.000000


In [43]:
# Save soft skills TF-IDF
transposed_tfidf_df.to_csv("reports/prg_name_tfidf.csv")